# Create `openalex.works_legacy` tables

Creates legacy mapping tables from `openalex.authors.work_authorships_backfill_moderated`:
1. `raw_affiliation_strings` - maps work/author to raw affiliation strings
2. `work_authors` - maps work to authors with is_corresponding flag

In [ ]:
%run ../utils/variables

### Step 1: Create raw_affiliation_strings table

In [ ]:
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works_legacy.raw_affiliation_strings') AS
WITH backfill_exploded AS (
    -- Explode authorships to get each author
    SELECT 
        paper_id AS work_id,
        author_sequence,
        authorship.raw_author_name,
        authorship.affiliations
    FROM openalex.authors.work_authorships_backfill_moderated
    LATERAL VIEW POSEXPLODE(authorships) t AS author_sequence, authorship
),
backfill_with_affiliations AS (
    -- Explode affiliations to get each affiliation per author
    SELECT 
        work_id,
        author_sequence,
        raw_author_name,
        affiliation.raw_affiliation_string
    FROM backfill_exploded
    LATERAL VIEW OUTER EXPLODE(affiliations) t AS affiliation
)
SELECT DISTINCT
    work_id,
    author_sequence,
    raw_author_name,
    raw_affiliation_string,
    current_timestamp() AS created_at
FROM backfill_with_affiliations;

### Step 2: Create work_authors table

In [ ]:
CREATE OR REPLACE TABLE identifier('openalex' || :env_suffix || '.works_legacy.work_authors') AS
SELECT DISTINCT
    paper_id AS work_id,
    author_sequence,
    authorship.author_id,
    authorship.raw_author_name,
    COALESCE(authorship.is_corresponding, false) AS is_corresponding,
    current_timestamp() AS created_at
FROM openalex.authors.work_authorships_backfill_moderated
LATERAL VIEW POSEXPLODE(authorships) t AS author_sequence, authorship;